In [1]:
import nltk
import gensim
import numpy as np
import pandas as pd

import pickle

from nltk.corpus import gutenberg, brown, reuters, stopwords

from collections import Counter, defaultdict
from pprint import pprint
from functools import reduce
from operator import itemgetter
from itertools import product

from utils.nltk_utils import *
from utils.ngrams import *
from utils.eval import get_test_keywords
stop_words = stopwords.words('english')

In [2]:
import traceback
from typing import List, Tuple

from tqdm import tqdm

from utils.grammar_utils import tags_seq_to_symbols
from utils.io import Cache
from utils.ngrams import ngram2str


observed_tags = Cache.load_observed_tags()
terminal_rules = Cache.load_terminal_rules()


def parse_phrases(tt_ngrams) -> Tuple[List, List[List[Tuple]]]:
    global observed_tags, terminal_rules
    
    phrases = []
    phrases_types = []

    if observed_tags is None:
        observed_tags = dict()

    for tt_gram in tt_ngrams:
        symbols = tuple(tags_seq_to_symbols([tag
                                             for _, tag in tt_gram]))

        phrase = tt_gram

        # check if tags phrase has been already observed
        tags_str = ngram2str(symbols)

        if tags_str in observed_tags:
            if observed_tags[tags_str] is not None:
                phrases.append(phrase)
                phrases_types.append(observed_tags[tags_str])

            continue

        p_types_dict = terminal_rules.get(tags_str)

        if p_types_dict:
            p_type = max(p_types_dict, key=lambda k: p_types_dict[k])

            phrases.append(phrase)
            phrases_types.append(p_type)

            observed_tags[tags_str] = p_type

        else:
            observed_tags[tags_str] = None

#     try:
#         Cache.save_observed_tags(observed_tags)

#     except Exception as e:
#         traceback.print_exc(e)

    return phrases_types, phrases


# Preparing corpora

## Load data from NLTK 

In [3]:
sents = gutenberg.sents() + brown.sents() + reuters.sents()

In [4]:
sents = [tuple(s) for s in sents]

words = [w.lower() 
         for s in sents for w in s]

vocab = sorted(list(set(words)))

print('Sents:', len(sents))
print('Words:', len(words))
print('Vocab:', len(vocab))

Sents: 210608
Words: 5503894
Vocab: 87046


In [5]:
# creating indexes

# {sentence: sent_index}
sents_ind = {sents[i]: i for i in range(len(sents))}

# {word: word_index}
vocab_ind = {vocab[j]: j for j in range(len(vocab))}

## Create mapping from words to sentences

In [6]:
stemmer = Stemmer()

stems = [stemmer.stem(word) for word in vocab]

# {word_index: stem}
stems_map = {vocab_ind[word]: stem 
             for word, stem in zip(vocab, stems)}

print('Stems:', len(set(stems)))

Stems: 58858


In [7]:
stem2word = {stem: vocab[ind]
             for ind, stem in stems_map.items()}

## Word2Vec model

In [8]:
# loading
model = gensim.models.Word2Vec.load('data/w2v/CBOW_300_10_hs_all')

In [14]:
sents_stemmed = [[stems_map[vocab_ind[w.lower()]] for w in s] for s in sents]

In [33]:
model = gensim.models.Word2Vec(sentences=sents_stemmed, size=300, window=15, min_count=1, hs=1, negative=0)
model.save('data/w2v/CBOW_300_10_hs_all')

In [9]:
# testing

print(model.wv.most_similar(positive=[stemmer.stem('woman'), stemmer.stem('king')], negative=[stemmer.stem('man')]))
print(model.wv.similarity(stemmer.stem('campus'), stemmer.stem('dormitory')))
print(model.wv.doesnt_match([stemmer.stem(w) for w in "dormitory bring campus".split()]))

[('queen', 0.3553354740142822), ('daughter', 0.35418325662612915), ('esther', 0.3361766040325165), ('husband', 0.3246108889579773), ('absalom', 0.3200189769268036), ('mordecai', 0.31594496965408325), ('samaria', 0.3088769316673279), ('vashti', 0.30666205286979675), ('hebron', 0.298782080411911), ('selleth', 0.29791125655174255)]
0.20989265750352995
bring


In [10]:
w2v = model.wv

## Parsing corpora

In [11]:
# load

with open('data/cache/stems_phrases', mode='rb') as fp:
    stems_phrases = pickle.load(fp)

In [9]:
parser = NGramsParser()

sents_words_indexes = parser.parse_sents_tokens(sents)

In [147]:
# {stem: Set[sentence_ind]}
stems_phrases = defaultdict(dict)

for i, (s_words, words_indexes) in tqdm(enumerate(zip(sents, sents_words_indexes), start=1)):
    words_ttokens = nltk.pos_tag([w.lower() for w in s_words])
    
    tt_ngrams = [ngr
                 for i in range(2, 5 + 1) 
                 for ngr in n_grams(words_ttokens, i, words_indexes, pad_left=False)]

    types, phrases = parse_phrases(tt_ngrams)

    # format and store phrases
    for t, p in zip(types, phrases):
        phrase_inds = tuple(vocab_ind[token] for token, _ in p)
        
        for word_ind in phrase_inds:
            if vocab[word_ind] not in stop_words:
                stem = stems_map[word_ind]
                
                if not stems_phrases[stem].get(t):
                    stems_phrases[stem][t] = defaultdict(set)
                
                stem_phr_t = stems_phrases[stem][t]
                
                stem_phr_t[phrase_inds].add(sents_ind[s_words])

210608it [06:48, 515.84it/s]


In [12]:
def get_phrases(word, stemmed=True):
    if stemmed:
        stem = word
    else:
        stem = stemmer.stem(word)
        
    phrs = stems_phrases[stem]
    
    if not phrs:
        return {}
    
    phrases_ = defaultdict(set)
    
    for p_type, phr_dict in phrs.items():
        for phrase, sents in phr_dict.items():
            phrase_ = tuple(vocab[ind] for ind in phrase)
            
            phrases_[p_type].add(phrase_)
            
    return phrases_

stems_phrases['assum']

{'ADJP': defaultdict(set,
             {(5566, 8631): {53379},
              (6509, 8629): {155524},
              (8629, 8322): {100786, 136405, 136436},
              (8629, 10767): {142473},
              (8629, 14426): {129262, 163315, 166916, 188102},
              (8629, 31882): {128992},
              (8629, 40323): {107429, 118657, 129508, 132871, 133063, 133149},
              (8629, 77432): {103574,
               129208,
               129583,
               129699,
               130147,
               130238,
               130557,
               131722,
               133059,
               133415,
               134962,
               136578,
               143528,
               155524,
               164946},
              (10042, 8629): {122610},
              (16046, 8629): {136027},
              (22386, 8629): {163315, 166916},
              (28806, 8631): {78913},
              (54085, 8631): {188105},
              (54928, 8631): {131253},
              (75026, 8

In [149]:
# save

with open('data/cache/stems_phrases', mode='wb') as fp:
    pickle.dump(stems_phrases, fp)

## Count number of phrases

In [13]:
# data for analysis

phrases_count = defaultdict(dict)

phrases_lengths_count = defaultdict(int)
phrases_types_count = defaultdict(int)

stems_phrases_count = defaultdict(dict)

for stem, stem_phrases_dict in stems_phrases.items():
    for phrase_type, phrases_dict in stem_phrases_dict.items():
#         stem_phrase_type_count = sum([len(s_ids) for s_ids in phrases_dict.values()])
        for phrase_tuple, s_ids in phrases_dict.items():
            phrase_len = len(phrase_tuple)
            phrase_sents_count = len(s_ids)
            
            if not stems_phrases_count[stem].get(phrase_len):
                stems_phrases_count[stem][phrase_len] = defaultdict(int)
                
            # count number of such phrase type for given stem
            stems_phrases_count[stem][phrase_len][phrase_type] += phrase_sents_count 

            # count total number of such phrase type
            phrases_count[phrase_len][phrase_type] = phrases_count[phrase_len].get(phrase_type, 0) + phrase_sents_count

            # count total number of all phrases
            phrases_lengths_count[phrase_len] += phrase_sents_count
            phrases_types_count[phrase_type] += phrase_sents_count

all_phrases_count = sum(phrases_types_count.values())

In [14]:
phrases_count

defaultdict(dict,
            {2: {'ADJP': 288142,
              'ADVP': 418156,
              'NP': 1985844,
              'PP': 3328,
              'VP': 66991},
             3: {'ADJP': 44770,
              'ADVP': 31064,
              'NP': 1353053,
              'PP': 7554,
              'VP': 8215},
             4: {'ADJP': 4179, 'ADVP': 223, 'NP': 429226},
             5: {'ADJP': 245, 'NP': 53025}})

In [15]:
WORD = 'strive'

stems_phrases_count[stemmer.stem(WORD)]

{2: defaultdict(int, {'ADJP': 12, 'ADVP': 8, 'NP': 28, 'VP': 9}),
 3: defaultdict(int, {'ADJP': 1, 'NP': 12, 'VP': 1}),
 4: defaultdict(int, {'NP': 4})}

In [16]:
def phrases_model(word, stemmed=True):
    """
        Return sorted log probs of phrase types and length for the word 
    """
    if not stemmed:
        stem = stemmer.stem(word)
    else:
        stem = word
    
    if not stems_phrases_count.get(stem):
        return []
    
    stem_phrases = stems_phrases_count[stem]
    
    phrases_probs = []
    
    for p_length, p_types_count in stem_phrases.items():
        for p_type, p_type_count in p_types_count.items():
            prob_len = phrases_lengths_count[p_length] / all_phrases_count
            
            prob_len_type = phrases_count[p_length][p_type] / phrases_lengths_count[p_length]
            
            prob_word_len_type = p_type_count / phrases_count[p_length][p_type]
            
            log_prob = sum(np.log(p) for p in [prob_word_len_type, prob_len_type, prob_len])
            
            phrases_probs.append(((p_type, p_length), log_prob))
    
    phrases, scores = [], []
    for phr, s in sorted(phrases_probs, key=itemgetter(1), reverse=True):
        phrases.append(phr)
        scores.append(s)
    
#     scores = np.exp(scores)
#     scores /= np.max(scores)
    
    return list(zip(phrases, scores))

phrases_model(WORD, stemmed=False)

[(('NP', 2), -12.029594340781607),
 (('ADJP', 2), -12.87689220116881),
 (('NP', 3), -12.876892201168811),
 (('VP', 2), -13.164574273620593),
 (('ADVP', 2), -13.282357309276973),
 (('NP', 4), -13.97550448983692),
 (('VP', 3), -15.361798850956813),
 (('ADJP', 3), -15.361798850956813)]

## Scoring phrases (experiments)

In [17]:
LENGTH_PEN = {
    2: 1.75,
    3: 1.25,
    4: 0.75,
    5: 0.25
}

REVERSE = False
SCORE_SIGN = -1
TOP_N = 5

P_TYPE_SCORE = dict(phrases_model(WORD, stemmed=False))

phrases_ = get_phrases(WORD, stemmed=False)

all_scored_phrases = defaultdict(dict)

for p_type, phrases in phrases_.items():
    phrases_ = list(phrases)
    
    scores = np.array(model.score([[stemmer.stem(w) for w in phr] 
                                   for phr in phrases_]))

    for phr, sc in zip(phrases_, scores):
        if not all_scored_phrases[p_type].get(len(phr)):
            all_scored_phrases[p_type][len(phr)] = []
        
        score = SCORE_SIGN * (sc + P_TYPE_SCORE[(p_type, len(phr))]) * LENGTH_PEN[len(phr)] 
        
        all_scored_phrases[p_type][len(phr)].append((phr, score))

In [18]:
for p_type, scored_phrases in all_scored_phrases.items():
    print('%s\n' % p_type)
    for n in scored_phrases:
        nps = scored_phrases[n]
        best_phr = sorted(nps, key=itemgetter(1), reverse=REVERSE)[:TOP_N]

        best_phr = [(' '.join(p), s) for p, s in best_phr]

        print(n)
        pprint(best_phr)
        print()

ADVP

2
[('strive against', 47.570972129232146),
 ('strive with', 47.65035311830685),
 ('strive for', 56.87991182459042),
 ('striving after', 60.76540462625668),
 ('strive at', 72.63408748758481)]

VP

2
[('striving against', 47.36485181683348),
 ('striving with', 47.444232805908186),
 ('striving for', 56.673791512191755),
 ('strive not', 57.704012020767195),
 ('striving in', 58.118454083145124)]

3
[('strive or fly', 53.73731564514743)]

NP

2
[('strive against', 45.37863693436525),
 ('also strive', 45.6956552048578),
 ('thou strive', 51.02504954537294),
 ('economic strivings', 51.26184067925355),
 ('the strivings', 52.57374654015443)]

3
[('the time striving', 42.21950544921492),
 ('no man strive', 42.35136284438704),
 ('dost thou strive', 44.11854281035994),
 ('a striving after', 44.736132760921464),
 ('a critic striving', 47.06441893188094)]

4
[('social and economic strivings', 26.75404972739356),
 ('than a critic striving', 32.95443148322242),
 ('the potsherd strive with', 34.645

In [19]:
# group different length together

candidate_phrases = defaultdict(list)

for p_type, scored_phrases in all_scored_phrases.items():
    for n in scored_phrases:
        nps = scored_phrases[n]
        best_phr = sorted(nps, key=itemgetter(1), reverse=REVERSE)[:TOP_N]

        best_phr = [(' '.join(p), s) for p, s in best_phr]
        
        candidate_phrases[p_type].extend(best_phr)
        
    # TODO: filter similar phrases and phrases with keywords
    candidate_phrases[p_type] = sorted(candidate_phrases[p_type], key=itemgetter(1), reverse=REVERSE)[:TOP_N]
    
candidate_phrases

defaultdict(list,
            {'ADJP': [('also strive', 47.178426460535405),
              ('so striving', 51.986163334162114),
              ('so strive', 51.986163334162114),
              ('not always strive', 52.29377697937595),
              ('vainly striving', 56.30455656460035)],
             'ADVP': [('strive against', 47.570972129232146),
              ('strive with', 47.65035311830685),
              ('strive for', 56.87991182459042),
              ('striving after', 60.76540462625668),
              ('strive at', 72.63408748758481)],
             'NP': [('social and economic strivings', 26.75404972739356),
              ('than a critic striving', 32.95443148322242),
              ('the potsherd strive with', 34.64571375556129),
              ('one powerful nation strives', 36.17675871558326),
              ('the time striving', 42.21950544921492)],
             'VP': [('striving against', 47.36485181683348),
              ('striving with', 47.444232805908186),
              

## Implementation of scoring phrases 

In [51]:
# TODO: add probabilistic CFG

LENGTH_PEN = {
    2: 1.75,
    3: 1.25,
    4: 0.75,
    5: 0.5
}

REVERSE = False
SCORE_SIGN = -1
TOP_N = 10

def get_scored_phrases(word, include_scores=True):
    P_TYPE_SCORE = dict(phrases_model(word, stemmed=False))

    phrases_ = get_phrases(word, stemmed=False)

    all_scored_phrases = defaultdict(dict)

    for p_type, phrases in phrases_.items():
        phrases_ = list(phrases)

        scores = np.array(model.score([[stemmer.stem(w) for w in phr] 
                                       for phr in phrases_]))

        for phr, sc in zip(phrases_, scores):
            if not all_scored_phrases[p_type].get(len(phr)):
                all_scored_phrases[p_type][len(phr)] = []

            score = SCORE_SIGN * (sc + P_TYPE_SCORE[(p_type, len(phr))]) * LENGTH_PEN[len(phr)] 

            all_scored_phrases[p_type][len(phr)].append((phr, score))

    # group different length together

    candidate_phrases = defaultdict(list)

    for p_type, scored_phrases in all_scored_phrases.items():
        for n in scored_phrases:
            nps = scored_phrases[n]
            best_phr = sorted(nps, key=itemgetter(1), reverse=REVERSE)[:TOP_N]

            best_phr = [(' '.join(p), s) for p, s in best_phr]

            candidate_phrases[p_type].extend(best_phr)

        # TODO: filter similar phrases and phrases with keywords
        candidate_phrases[p_type] = sorted(candidate_phrases[p_type], key=itemgetter(1), reverse=REVERSE)#[:TOP_N]
        
        if not include_scores:
            candidate_phrases[p_type] = [p for p, _ in candidate_phrases[p_type]]
            
    return candidate_phrases

## Working with keywords

In [21]:
kws = get_test_keywords('data/lingualeo_words.csv')

print('Keywords: ', len(kws))

Keywords:  874


In [41]:
TEST_SIZE = 5
np.random.seed = 0

all_kws_found = False

while not all_kws_found:    
    test_kws = list(np.random.choice(list(kws), size=TEST_SIZE, replace=False))

    word_kws = list(test_kws)
    
    print('Test keywords:', word_kws)
    
    test_kws = [stemmer.stem(kw) for kw in test_kws]
    kws_phrases = [stems_phrases.get(kw) for kw in test_kws]
    print('Test keywords stems:', test_kws)

    all_kws_found = all(kws_phrases)
    
    print('All kws found:', all_kws_found)
    print()
    

Test keywords: ['it', 'far', 'tremendous', 'frown', 'prosecution']
Test keywords stems: ['it', 'far', 'tremend', 'frown', 'prosecut']
All kws found: False

Test keywords: ['it', 'feed', 'income', 'astride', 'precedence']
Test keywords stems: ['it', 'feed', 'incom', 'astrid', 'preced']
All kws found: False

Test keywords: ['percolate', 'kin', 'firm', 'cut', 'equally']
Test keywords stems: ['percol', 'kin', 'firm', 'cut', 'equal']
All kws found: True



In [42]:
def cluster_keywords(kws):
    pairs = []

    for i, kw in enumerate(kws):
        cp = list(kws)
        cp.remove(kw)

        dsts = w2v.distances(kw, cp)
#         print(kw, dsts)
        max_ind = np.argmin(dsts)
#         print(i, max_ind)
        sim_kw = kws[max_ind + (1 if i <= max_ind else 0)]
#         print(kw, sim_kw)

        pair = (kw, sim_kw)

        if (sim_kw, kw) in pairs:
            pair = (sim_kw, kw)

        pairs.append(pair)

    final_clusters = []
    
    kws_ = list(kws)
    
    for kws_pair, cnt in Counter(pairs).items():
        if cnt > 1:
            final_clusters.append(kws_pair)
            [kws_.remove(w) for w in kws_pair]
        
    return final_clusters + [(kw,) for kw in kws_]

cluster_keywords(test_kws)

[('percol', 'kin'), ('firm',), ('cut',), ('equal',)]

In [80]:
for kws_tuple in cluster_keywords(test_kws):
    result = gen_sents_candidates(kws_tuple, [get_scored_phrases(kw, include_scores=False) for kw in kws_tuple])
    
    ranked_sents = rank_sents(result, test_kws)
    
    print(kws_tuple, len(ranked_sents))
    pprint(ranked_sents)
    print()

Skipped: 194 out of 197


UnboundLocalError: local variable 'job_no' referenced before assignment

In [79]:
sents_rules = Cache.load_terminal_rules_sents()

total_sents = sum(v['S'] for v in sents_rules.values())

def rank_sents(ss, kwss, top_n=20):
    res_sents = []
    
    sents_stems = [[stems_map.get(vocab_ind.get(w, w), w) 
                    for w in s.split(' ')]
                  for _, s in ss]
    
    scores = model.score(sents_stems)
    
    for (s_prob, s), score in zip(ss, scores):
        s_spl = s.split(' ')

        if len(s_spl) > 2:
            not_sw_prob = len([w
                           for w in s_spl
                           if w.lower() not in stop_words and stems_map[vocab_ind[w.lower()]] not in kwss]) / len(s_spl)
            
            print(score, np.log(s_prob or 1e-6), np.log(not_sw_prob or 1e-6))
            
            res_sents.append((s.capitalize() + '.', score + np.log(s_prob or 1e-6) + np.log(not_sw_prob or 1e-6)))
            
    return sorted(res_sents, key=itemgetter(1), reverse=True)[:top_n]
    
def combine_elements(*args):
    return list(product(*args, repeat=1))

def gen_sents_candidates(kws, kws_phrases):
    marks = {'COMMA': [","],
             'COLON': [":"],
             'SEMICOLON': [";"],
             # 'DOT': ["."],
             'QUESTION': ["?"],
             'EXCLAM': ["!"],
             'DASH': ["-"]}
    
    res_sents = set()

    if len(kws) == 1:
        kw = kws[0]
        phrs = kws_phrases[0]
        
        for p_type, p_type_sents in stems_phrases[kw].items():
            res_sents.update((0, ' '.join(sents[s_id])) for s_ids_set in p_type_sents.values() for s_id in list(s_ids_set) )
            
        return res_sents
    
    phrases = defaultdict(dict)

    for kw, kw_phrases in zip(kws, kws_phrases):
        for p_type in kw_phrases:
            phrases[p_type][kw] = kw_phrases[p_type]

    skipped = 0
    
    for sents_rule, sents_count in sents_rules.items():
        s_prob = max(sents_count['S'], 1000) / total_sents
        
        if s_prob < 0.05:
            skipped += 1
            continue
        
        sent_symbols = str2ngram(sents_rule)
        
        sent_phrases = []
        
        # form lists of phrases of each keyword
        for p_type in sent_symbols:
            if p_type not in marks:
                sent_phrases.append([(kw, phrases[p_type].get(kw))
                                     for kw in kws
                                    if phrases[p_type].get(kw)])
            else:
                sent_phrases.append((None, marks.get(p_type, [])))
            
        # combine phrases of keywords
        kws_phrases_product = combine_elements(*sent_phrases)
        
        # combine sents
        for kws_comb in kws_phrases_product:
            if not kws_comb is None:
                continue
        
            un_combs = [kw for kw, _ in kws_comb 
                        if kw is not None]
            if len(set(un_combs)) <= 1:
                continue
            
            kws_comb = [comb for _, comb in kws_comb]
            
            sents_candidates = [(s_prob, ' '.join(s))
                                for s in combine_elements(*kws_comb)]
            
            res_sents.update([s_cand for s_cand in sents_candidates])
    
    print('Skipped: %s out of %s' % (skipped, len(sents_rules)))
    return res_sents